# Linear Regression

### Generate dataset

In [10]:
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

### Read dataset

In [11]:
def load_array(data_arrays, batch_size, is_train=True):
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [12]:
next(iter(data_iter))

[tensor([[-1.2545, -0.0082],
         [-0.0443,  0.8248],
         [ 0.1954,  1.8973],
         [-1.0373, -2.2007],
         [ 1.0498,  0.5620],
         [ 1.2204, -0.8398],
         [ 0.3137, -0.5445],
         [ 1.1154,  0.1769],
         [-0.9834,  0.1374],
         [ 0.0551, -0.3457]]),
 tensor([[ 1.7166],
         [ 1.3109],
         [-1.8492],
         [ 9.5987],
         [ 4.3857],
         [ 9.4855],
         [ 6.6676],
         [ 5.8058],
         [ 1.7469],
         [ 5.4698]])]

### Define model

In [13]:
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

### Initialize model parameters

In [14]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

### Define the loss function

In [15]:
loss = nn.MSELoss()

### Define optimization algorithm

In [16]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

### Training

In [17]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epcoh {epoch + 1}, loss {l:f}')

epcoh 1, loss 0.000169
epcoh 2, loss 0.000099
epcoh 3, loss 0.000099


In [18]:
w = net[0].weight.data
print(f'Estimated error of w: {true_w - w.reshape(true_w.shape)}')
b = net[0].bias.data
print(f'Estimated error of b: {true_b - b}')

Estimated error of w: tensor([7.2002e-05, 7.1526e-06])
Estimated error of b: tensor([-3.5286e-05])
